# Deep Q-Learning

In this notebook, you will implement a deep Q-Learning reinforcement algorithm. The implementation borrows ideas from both the original DeepMind Nature paper and the more recent asynchronous version:<br/>
[1] "Human-Level Control through Deep Reinforcement Learning" by Mnih et al. 2015<br/>
[2] "Asynchronous Methods for Deep Reinforcement Learning" by Mnih et al. 2016.<br/>

In particular:
* We use separate target and Q-functions estimators with periodic updates to the target estimator. 
* We use several concurrent "threads" rather than experience replay to generate less biased gradient updates. 
* Threads are actually synchronized so we start each one at a random number of moves.
* We use an epsilon-greedy policy that blends random moves with policy moves.
* We taper the random search parameter (epsilon) and the learning rate to zero during training.

This gives a simple and reasonably fast general-purpose RL algorithm. We use it here for the Cartpole environment from OpenAI Gym, but it can easily be adapted to others. For this notebook, you will implement 4 steps:

1. The backward step for the Q-estimator
2. The $\epsilon$-greedy policy
3. "asynchronous" initialization 
4. The Q-learning algorithm

To get started, we import some prerequisites.

In [1]:
import BIDMat.{CMat,CSMat,DMat,Dict,FMat,FFilter,Filter,GFilter,GMat,GDMat,GIMat,GLMat,GSMat,GSDMat,HMat,IDict,Image,IMat,LMat,Mat,SMat,SBMat,SDMat,TMat}
import BIDMat.MatFunctions._
import BIDMat.SciFunctions._
import BIDMat.Solvers._
import BIDMat.JPlotting._
import BIDMach.Learner

import BIDMach.models.{Click,FM,GLM,KMeans,KMeansw,LDA,LDAgibbs,Model,NMF,SFA,RandomForest,SVD}
import BIDMach.networks.{Net}
import BIDMach.datasources.{DataSource,MatSource,FileSource,SFileSource}
import BIDMach.datasinks.{DataSink,MatSink}
import BIDMach.mixins.{CosineSim,Perplexity,Top,L1Regularizer,L2Regularizer}
import BIDMach.updaters.{ADAGrad,Batch,BatchNorm,Grad,IncMult,IncNorm,Telescoping}
import BIDMach.causal.{IPTW}
import BIDMach.rl.ALE

Mat.checkMKL(false)
Mat.checkCUDA
Mat.setInline


1 CUDA device found, CUDA version 8.0


The block below lists some parameters you can tune. Most should be self-explanatory. They are currently set to train Pong.

In [2]:
val nsteps = 2000001                     // Number of steps to run (game actions per environment)
val npar = 16                            // Number of parallel environments
val target_window = 50                   // Interval to update target estimator from q-estimator
val discount_factor = 0.99f              // Reward discount factor
val printsteps = 10000                   // Number of steps between printouts
val render = false                       // Whether to render an environment while training

val epsilon_start = 0.3f                 // Parameters for epsilon-greedy policy: initial epsilon
val epsilon_end = 0.1f                   // Final epsilon
val neps = (0.9*nsteps).toInt            // Number of steps to decay epsilon

//val learning_rate = 1e-5f
val learning_rate = 3e-6f                // Initial learning rate
val lr_end = learning_rate               // Final learning rate
val nlr = neps                           // Steps to decay learning rate
val gsq_decay = 0.99f                    // Decay factor for MSProp
val vel_decay = 0.9f                     // Momentum decay
val gclip = 1f                           // gradient clipping
val rmsevery = 5                         // apply gradient after skipping this many steps

val nhidden = 16                         // Number of hidden layers for estimators
val nhidden2 = 32
val nhidden3 = 256

val init_moves = 10000                   // Upper bound on random number of moves to take initially
val nwindow = 4                          // Sensing window = last n images in a state

// The epsilon and learning rate decay schedules
// val epsilons = np.linspace(epsilon_start, epsilon_end, neps)

val epsilons = epsilon_start / (1f + row(0->neps)/(neps*epsilon_end/epsilon_start))
val learning_rates = learning_rate - row(0 -> nlr) * ((lr_end - learning_rate) / nlr)


// Model type and action definitions

val game_bin="/code/ALE/roms/Pong.bin"                 
val VALID_ACTIONS = irow(0, 3, 4)
val nactions= VALID_ACTIONS.length
val nfeats = 80*80  
val height = 80
val width = 80


80

Below are environment-specific parameters. The function "preprocess" should process an observation returned by the environment into a vector for training. For CartPole we simply append a 1 to implement bias in the first layer. 

For visual environments you would typically crop, downsample to 80x80, set color to a single bit (foreground/background), and flatten to a vector. 

*nfeats* is the dimension of the vector output by *preprocess*.

In [3]:

def preprocess(I:Array[Byte], out:FMat):FMat = {
//  Preprocess Pong game frames into vectors.
//  Input:
//    - (3,160,210) uint8 frame representing Pong game screen.
//  Returns:
//    - Downsampled (DxD) matrix of 0s and 1s.
    var i = 0;
    val res = if (out.asInstanceOf[AnyRef] == null) zeros(width\height\1\1) else out;
    while (i < 80) {
        var j = 0;
        while (j < 80) {
            val x = I(j*2 + 160 * (i*2 + 35));
            res.data(j + 80*i) = {if (x == 34) 0f else {if (x != 0) 1f else 0f}};
            j += 1;
        }
        i += 1;
    }
    res
}


Here is the Q-estimator class. We use two instances of this class, one for the target estimator, and one for the Q-estimator. The Q function is normally represented as a scalar $Q(x,a)$ where $x$ is the state and $a$ is an action. For ease of implementation, we actually estimate a vector-valued function $Q(x,.)$ which returns the estimated reward for every action. The model here has just a single hidden layer:

<pre>
Input Layer (nfeats) => FC Layer => RELU => FC Layer => Output (naction values)
</pre>

## 1. Implement Q-estimator gradient
Your first task is to implement the
<pre>Estimator.gradient(s, a, y)</pre>
method for this class. **gradient** should compute the gradients wrt weight arrays W1 and W2 into
<pre>self.grad['W1']
self.grad['W2']</pre>
respectively. Both <code>a</code> and <code>y</code> are vectors. Be sure to update only the output layer weights corresponding to the given action vector. 

In [4]:

import jcuda.jcudnn.cudnnTensorFormat._
import jcuda.jcudnn.cudnnConvolutionMode._

val convt = CUDNN_CROSS_CORRELATION;
val tformat = Net.TensorNCHW;

class Myopts extends Net.Options with ADAGrad.Opts;

class Estimator(nhidden:Int, nhidden2:Int, nhidden3:Int, nactions:Int) {
    
    val opts = new Myopts;  
    opts.vexp = 1f;
    opts.texp = 0f;
    opts.waitsteps = -1;
    opts.vel_decay = vel_decay;
    opts.gsq_decay = gsq_decay;
    opts.tensorFormat = tformat;
    
    import BIDMach.networks.layers.Node._;

    val in =    input;
    val acts =  input;
    
    val conv1 = conv(in)(w=7,h=7,nch=nhidden,stride=4,pad=3,initv=1f,convType=convt);
    val relu1 = relu(conv1);

    val conv2 = conv(relu1)(w=3,h=3,nch=nhidden2,stride=2,pad=1,convType=convt);
    val relu2 = relu(conv2);

    val fc3 =   linear(relu2)(outdim=nhidden3,initv=2e-2f);
    val relu3 = relu(fc3);
    
    val fc4 =   linear(relu3)(outdim=nactions,initv=5e-2f); 
    val npredict = 9;                                       // Number of layers for action prediction. 
    
    val sel =   fc4(acts);
    
    val out =   glm(sel)(izeros(1,1))

    val nodes = (in     \ acts   on
                 conv1  \ relu1  on
                 conv2  \ relu2  on
                 fc3    \ relu3  on
                 fc4    \ null   on
                 sel    \ out    ).t

    opts.nodemat = nodes;
    
    val net = new Net(opts);
    val adagrad = new ADAGrad(opts);
    
    def formatStates(s:FMat):FMat = {
        if (tformat == Net.TensorNCHW) {
            s.reshapeView(nwindow\height\width\npar);
        } else {
            s.reshapeView(height\width\nwindow\npar).transpose(2\0\1\3);
        }
    }
    
/** Perform the initialization that is normally done by the Learner */

    var initialized = false;
    
    def checkinit(states:FMat, actions:IMat, rewards:FMat) = {
    if (net.mats.asInstanceOf[AnyRef] == null) {
        net.mats = new Array[Mat](3);
        net.gmats = new Array[Mat](3);
    }
    net.mats(0) = states;
    if (net.mats(1).asInstanceOf[AnyRef] == null) {
        net.mats(1) = izeros(1, states.ncols);              // Dummy action vector
        net.mats(2) = zeros(1, states.ncols);               // Dummy reward vector
    }
    if (actions.asInstanceOf[AnyRef] != null) {
        net.mats(1) <-- actions;
    }
    if (rewards.asInstanceOf[AnyRef] != null) {
        net.mats(2) <-- rewards;
    }
    if (!initialized) {
        net.useGPU = (opts.useGPU && Mat.hasCUDA > 0);
        net.init();
        adagrad.init(net);
        initialized = true;
    }
    net.copyMats(net.mats, net.gmats);
    net.assignInputs(net.gmats, 0, 0);
    net.assignTargets(net.gmats, 0, 0);
    }
    
/**  Run the model forward given a state as input up to the action prediction layer. 
     Action selection/scoring layers are not updated.
     returns action predictions */
    
    def predict(states:FMat):FMat = {
        val fstates = formatStates(states);
        checkinit(fstates, null, null);
        for (i <- 0 until npredict) net.layers(i).forward;
        FMat(net.layers(npredict-1).output);
    }

/** Run the model all the way forward to the squared loss output layer, 
    and then backward to compute gradients.
    An action vector and reward vector must be given. */  
    
    def gradient(states:FMat, actions:IMat, rewards:FMat):Float = {
        val fstates = formatStates(states);
        checkinit(fstates, actions, rewards);
        net.forward;
        net.backward(0, 0);
        val dout = net.layers(net.layers.length-1).score;
        sqrt(-dout/dout.length).fv                             // return the RMS residual error. 
    }
        
/** MSprop, i.e. RMSprop without the square root, or natural gradient */
        
    def msprop(learning_rate:Float) = {                
        opts.lrate = learning_rate;
        adagrad.update(0,0,0);
        net.cleargrad;
    }

    val aselector = irow(0->npar)*nactions;
    val A = zeros(nactions, npar);

    def policy(state:FMat, epsilon:Float):FMat = {
//    """ Take an estimator and state and predict the best action.
//    For each input state, return a vector of action probabilities according to an epsilon-greedy policy"""
        A.set(epsilon / nactions);
        val q_values = predict(state);
        val (_,best_action) = maxi2(q_values)
        A(best_action + aselector) = A(best_action + aselector) + (1f - epsilon)
        A
    }

    def update_estimator(to_estimator:Estimator, window:Int, istep:Int) = {
//    """ every <window> steps, Copy model state from from_estimator into to_estimator"""
        if (istep % window == 0) {
            for (k  <- 0 until net.modelmats.length) {
                to_estimator.net.modelmats(k) <-- net.modelmats(k);
            }
        }
    }

};



## 3. Implement "Asynchronous Threads"

Don't try that in Python!! Actually all we do here is create an array of environments and advance each one a random number of steps, using random actions at each step. Later on we will make *synchronous* updates to all the environments, but the environments (and their gradient updates) should remain uncorrelated. This serves the same goal as asynchronous updates in paper [2], or experience replay in paper [1].

In [5]:

// Initialize the games
print("Initializing games...")
val envs = new Array[ALE](npar)
val state = zeros(width\height\nwindow\npar)
val img0 = zeros(width\height\1\1)
var total_time=0f
var total_steps=0
var total_epochs = 0

import java.util.Random
val rn = new Random

tic
for (i <- 0 until npar) {
    envs(i) = new ALE
    envs(i).setInt("random_seed", i)
    envs(i).loadROM(game_bin)

    val nmoves = rn.nextInt(init_moves - nwindow) + nwindow
    for (j <- 0 until nmoves) {   
        val action = VALID_ACTIONS(rn.nextInt(nactions))
        val (obs, reward, done) = envs(i).step2(action)
        total_steps += 1;
        if (nmoves - j <= nwindow) {
            val k = nwindow - nmoves + j;
            state(?, ?, k->(k+1), i) = preprocess(obs, img0)
        }
        if (done) {
            envs(i).reset() 
            total_epochs += 1
        }
    }
}

total_time = toc     
println("%d steps, %d epochs in %5.4f seconds at %5.4f msecs/step" format(
    total_steps, total_epochs, total_time, 1000f*total_time/total_steps))


Initializing games...75791 steps, 58 epochs in 81.1140 seconds at 1.0702 msecs/step


Select integer actions using the probability distribution in each column of <code>probs</code>

In [6]:
def random_choices(probs:FMat):IMat = {
    val result = izeros(1, probs.ncols);
    var i = 0;
    while (i < probs.ncols) {
        val r = rn.nextFloat();
        var j = 0;
        var cumprob = probs(0, i);
        while (r > cumprob && j+1 < probs.length) {
            j += 1;
            cumprob += probs(j, i);
        }
        result(i) = j;
        i += 1
    }
    result
}

## 4. Implement Deep Q-Learning
In this cell you actually implement the algorithm. We've given you comments to define all the steps. You should also add book-keeping steps to keep track of the loss, reward and number of epochs (where env.step() returns done = true). 

In [ ]:
// Create estimators
val q_estimator = new Estimator(nhidden, nhidden2, nhidden3, nactions)
val target_estimator = new Estimator(nhidden, nhidden2, nhidden3, nactions)

// Initialize them by making predictions
q_estimator.predict(state);
target_estimator.predict(state);

tic
var block_loss = 0f
var block_reward = 0f
var total_reward = 0f
total_epochs = 0
var last_epochs = 0
val new_state = state.copy
Mat.useGPUcache = true;

val times = zeros(1,8);
val dtimes = zeros(1,7);
val rand_actions = int(rand(1, npar) * nactions);
val (obs0, rewards0, dones0) = ALE.stepAll2(envs, VALID_ACTIONS(rand_actions));           // step through parallel envs
var actions:IMat = null;
var action_probs:FMat = null;

for (istep <- 0 until nsteps) {
//    if (render): envs[0].render()
    times(0) = toc
    val epsilon = epsilons(math.min(istep, neps-1));                                // get an epsilon for the eps-greedy policy
    val lr = learning_rates(math.min(istep, nlr-1));                                // update the decayed learning rate
    
    q_estimator.update_estimator(target_estimator, target_window, istep);          // update the target estimator if needed    
    times(1) = toc
    
    action_probs = q_estimator.policy(state, epsilon);                         // get the next action probabilities from the policy
    times(2) = toc
                                                 
    actions = random_choices(action_probs);                                     // Choose actions using the policy
    val (obs, rewards, dones) = ALE.stepAll2(envs, VALID_ACTIONS(actions), obs0, rewards0, dones0)           // step through parallel envs
    times(3) = toc
    
    for (i <- 0 until npar) {                                                     
        val img = preprocess(obs(i), img0);                                               // process the observation
        new_state(?,?,0->(nwindow-1),i) = state(?,?,1->nwindow,i);              // shift the image stack and add a new image
        new_state(?,?,nwindow-1,i) = img;         
    }    
    total_epochs += sum(dones).v.toInt
    block_reward += sum(rewards).v  
    times(4) = toc
    
    val q_values_next = target_estimator.predict(new_state);                        // predict the Q values
    times(5) = toc
    
    dones <-- (dones + (rewards != 0f) > 0f);
    val targets = rewards+discount_factor*(1f-dones) *@ maxi(q_values_next);        // compute target values
    block_loss += q_estimator.gradient(state, actions, targets);                    // compute q-estimator gradient and return the loss
    times(6) = toc
     
    if (istep % rmsevery == 0) {
       q_estimator.msprop(lr);                       // apply the gradient update
//        print("ds1=%f, ss1=%f, ds2=%f, ss2=%f\n" format(res(0,0), res(1,0), res(0,1), res(1,1)));
    }
    times(7) = toc
    
    dtimes ~ dtimes + (times(0,1->8) - times(0,0->7))
    val t = toc;
    if (istep % printsteps == 0) {
        total_reward += block_reward;
        println("step %d, time %2.1f, loss %9.8f, epochs %d, reward/epoch %6.5f, cum reward/epoch %6.5f" format(
                istep, t, block_loss/printsteps, total_epochs, block_reward/math.max(1,total_epochs-last_epochs), total_reward/math.max(1,total_epochs)))
        last_epochs = total_epochs;
        block_reward = 0f;
        block_loss = 0f;
    }
    state <-- new_state;
    //    Mat.debugMem = true;
    //    Mat.debugMemThreshold = 0;
}
dtimes



step 0, time 0.0, loss 0.00000162, epochs 0, reward/epoch 0.00000, cum reward/epoch 0.00000
step 10000, time 110.2, loss 0.06295285, epochs 145, reward/epoch -20.38621, cum reward/epoch -20.38621
step 20000, time 219.2, loss 0.04919356, epochs 281, reward/epoch -20.51471, cum reward/epoch -20.44840
step 30000, time 327.9, loss 0.05800981, epochs 387, reward/epoch -19.32076, cum reward/epoch -20.13953
step 40000, time 436.2, loss 0.05846595, epochs 478, reward/epoch -19.59341, cum reward/epoch -20.03556
step 50000, time 544.4, loss 0.06574187, epochs 561, reward/epoch -18.80723, cum reward/epoch -19.85383
